# Sample Size Calculator

[How to use](https://tubitv.atlassian.net/wiki/spaces/DST/pages/2045116757/TDR+Sample+Size+Calculator)

## Specify your settings

In [ ]:
import tubi_data_runtime as tdr
import math
import pandas as pd
import numpy as np
from datetime import date
import time
import itertools
import threading
import sys

import ipywidgets as widgets
from ipywidgets import interact, interactive, FloatSlider, fixed, interact_manual, SelectMultiple, Button, Checkbox, Output, Dropdown, Accordion, Tab, HBox, VBox, Layout, HTML
from IPython.display import clear_output, display as ipy_display
from traitlets import traitlets

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

from ssc_utils.filter_generator import filter_generator
from ssc_utils.raw_user_data import raw_user_data
from ssc_utils.metric_switcher import metric_switcher
from ssc_utils.metric_summary import metric_summary
from ssc_utils.cuped import cuped
import ssc_utils.calculator as c

# load choices
event_name_choices = filter_generator().event_name_choices()
filter_metrics_choices = filter_generator().filter_metrics_choices()

In [ ]:
# function for checking that only and exactly one of ALLOCATION, DAYS, or EFFECT_SIZE_RELATIVE is empty
def check_alloc_and_days(allocation, days, effect):
    "xor for `allocation`, `days`, `effect` parameters"
    return np.sum([bool(allocation.result), bool(days.result), bool(effect.result)]) == 2

alloc_days_error_msg = 'ERROR: One (and only one) of "allocation", "days", or "effect" must be blank!'

In [ ]:
# add limits on values here? e.g., allocation has to be between 0 and 1 inclusive, days has to be > 0, etc
ALLOCATION = interactive(c.allocation, allocation='1.00')
DAYS = interactive(c.days, days='')
EFFECT_SIZE_RELATIVE = interactive(c.effect, effect='0.0100')
NUMBER_VARIATIONS = interactive(c.treatments, treatments='1')


metrics_total = metric_switcher().possible_metrics()
primary_metric = widgets.Dropdown(
    options=metrics_total['metric_clean'].tolist(),
    description='metric',
    value='tvt-capped')

attribute_filter = interactive(filter_generator().make_sql_condition_string, 
                               field = filter_generator().filter_attributes_choices(), 
                               condition = filter_generator().condition_choices(), 
                               value = '', 
                               filter_type = fixed('attribute'))

metric_filter = interactive(filter_generator().make_sql_condition_string, 
                            field = filter_metrics_choices, 
                            condition = filter_generator().condition_choices(), 
                            value = '', 
                            filter_type = fixed('metric'))


primary_event = SelectMultiple(
    options = event_name_choices,
    value = ('no event filter',), 
    description='event'
#     disabled=False
)

primary_event_sub_cond = interactive(filter_generator().make_sql_condition_string, 
                                     field = filter_generator().event_sub_cond_field_choices(), 
                                     condition = filter_generator().condition_choices(), 
                                     value = '', 
                                     filter_type = fixed('event'))


pre_event = SelectMultiple(
    options = event_name_choices,
    value = ('no event filter',), 
    description='event',
    disabled=False
)

pre_event_sub_cond = interactive(filter_generator().make_sql_condition_string, 
                                 field = filter_generator().event_sub_cond_field_choices(), 
                                 condition = filter_generator().condition_choices(), 
                                 value = '', 
                                 filter_type = fixed('event'))


time_interval = interactive(filter_generator().interval, interval = 'NULL')

In [ ]:
# set up "objects"

## parameters tab
separator_label = widgets.HTML(value='<br><b>Enter 2 out of 3 (one field must be empty):</b>')
parameters = VBox([NUMBER_VARIATIONS, primary_metric, separator_label, ALLOCATION, EFFECT_SIZE_RELATIVE, DAYS])

## filters tab
filter_accordion = Accordion([attribute_filter, metric_filter])
filter_accordion_titles = ['attribute', 'metric']
for i, title in enumerate(filter_accordion_titles):
    filter_accordion.set_title(i, title)

## event filters tab
primary_event_set = VBox([primary_event, primary_event_sub_cond])
pre_event_set = VBox([pre_event, pre_event_sub_cond, time_interval])

event_filter_accordion = Accordion([primary_event_set, pre_event_set])
event_filter_accordion_titles = ['primary event','pre event']
for i, title in enumerate(event_filter_accordion_titles):
    event_filter_accordion.set_title(i, title)


## combine everything together into multiple tabs
list_widgets  = [
    parameters,
    filter_accordion,
    event_filter_accordion   
]

tab = Tab(children = list_widgets)

titles = ['parameters','filters','event filters']
for i, title in enumerate(titles):
    tab.set_title(i, title)
tab

In [ ]:
def concat_3child_filters(filt):
    if filt.children[0].value != 'no filters':
        return filt.children[0].value + ' ' + filt.children[1].value + ' ' + filt.children[2].value
    else: 
        return 

    
filter_output = Output()
show_filter_button = Button(description="Show all filters", layout=Layout(width='200px'))
ipy_display()


def print_filters_on_button_clicked(b):
    filter_output.clear_output(wait = True)
    with filter_output:
        if check_alloc_and_days(allocation=ALLOCATION, days=DAYS, effect=EFFECT_SIZE_RELATIVE):
            print('parameters selected:')
            print('\t number of variations', float(NUMBER_VARIATIONS.result))
            if ALLOCATION.result:
                print('\t allocation', float(ALLOCATION.result))
            if DAYS.result:
                print('\t days', float(DAYS.result))
            if EFFECT_SIZE_RELATIVE.result:
                print('\t effect size', float(EFFECT_SIZE_RELATIVE.result))
                
            print()
            print('metric selected:', primary_metric.value)

            print()
            print('filters selected:')
            print('\t', concat_3child_filters(attribute_filter))
            print('\t', concat_3child_filters(metric_filter))

            print()
            print('event filters selected:')
            if primary_event.value[0] != 'no event filter': 
                print('\t', 'primary event:', primary_event.value[0], ';', concat_3child_filters(primary_event_sub_cond))
            else: print('\t', None)
            if pre_event.value[0] != 'no event filter': 
                print('\t', 'pre event:', pre_event.value[0], ';', concat_3child_filters(pre_event_sub_cond), )
                if time_interval.children[0].value != 'NULL':
                    print('time interval between events:', time_interval.children[0].value)
        else:
            print(alloc_days_error_msg)

In [ ]:
apply_output = Output()
apply_button = Button(description="Apply filters", layout=Layout(width='200px'))

def apply_on_button_clicked(b):
    global FINAL_SQL

    filters_sql = filter_generator().generate_filter_cte(attribute_condition_interact = attribute_filter, 
                                                         metric_condition_interact = metric_filter, 
                                                         event1_condition_interact = pre_event, 
                                                         event1_sub_condition_interact = pre_event_sub_cond, 
                                                         event2_condition_interact = primary_event, 
                                                         event2_sub_condition_interact = primary_event_sub_cond, 
                                                         event_time_interval_interact = time_interval)
    raw_user_sql = raw_user_data().generate_raw_user_data_cte(filters_sql, primary_metric.value)
    user_sql = metric_switcher().generate_user_data_cte(primary_metric.value, metrics_total) 
    summary_sql = metric_summary().generate_metric_summary_cte() 
    cuped_sql = cuped().generate_cuped_cte(event2_condition_interact = primary_event)

    FINAL_SQL =  filters_sql + raw_user_sql + user_sql + summary_sql + cuped_sql
    
ipy_display(apply_button, apply_output)
apply_button.on_click(apply_on_button_clicked)

## Results

In [ ]:
def speak(text):
    from IPython.display import Javascript as js, clear_output
    # Escape single quotes
    text = text.replace("'", r"\'")
    display(js(f'''
    if(window.speechSynthesis) {{
        var synth = window.speechSynthesis;
        synth.speak(new window.SpeechSynthesisUtterance('{text}'));
    }}
    '''))
    # Clear the JS so that the notebook doesn't speak again when reopened/refreshed
    clear_output(False)

def animate():
    steps = ['.  ', '.. ', '...']
    t = 0
    
    for c in itertools.cycle(steps):
        if done:
            break
        
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        
        sys.stdout.write('\rRunning' + c + ' ' + timer)
        sys.stdout.flush()
        time.sleep(1)
        t += 1
    sys.stdout.write('\rDone! Elapsed time: ' + timer)

In [ ]:
run_layout = Layout(width='300px')
run_controls = Output(layout=run_layout)
output = Output()
speak_checkbox = Checkbox(
    value=False,
    description='Announce when execution completes',
    disabled=False,
    indent=False,
    layout=run_layout
)
run_button = Button(description="Calculate sample size", layout=run_layout)
run_button.style.button_color = 'lightgreen'

run_controls.append_display_data(speak_checkbox)
run_controls.append_display_data(run_button)


def run_on_button_clicked(b):
    global done
    
    output.clear_output(wait = True)
    with output:
        if check_alloc_and_days(allocation=ALLOCATION, days=DAYS, effect=EFFECT_SIZE_RELATIVE):
            
            done = False
            t = threading.Thread(target=animate)
            t.start()
            
            # Calculating below
            raw_df = tdr.query_redshift(FINAL_SQL)
            final_df = c.calculate_sample_required(df = raw_df, 
                                                   effect_size_relative = EFFECT_SIZE_RELATIVE, 
                                                   number_variations = NUMBER_VARIATIONS, 
                                                   allocation = ALLOCATION,
                                                   days = DAYS,
                                                   power = 0.8, 
                                                   alpha = 0.05)
            
            if (speak_checkbox.value):
                speak('Sample Size Calculator has finished running.')
            display(final_df.sort_values('platform').style.hide().format(precision=4))
            
            # Calculating done
            done = True
            
        else:
            print(alloc_days_error_msg)

In [ ]:
# output SQL for debugging purposes; can copy and manually run this elsewhere

sql_output = Output()
print_button = Button(description="Print SQL (for debugging)", layout=Layout(width='200px'))

def print_sql_on_button_clicked(b):
    sql_output.clear_output(wait = True)
    with sql_output:
        print(FINAL_SQL)

In [ ]:
ipy_display(run_controls, output)
run_button.on_click(run_on_button_clicked)

## Show filters

In [ ]:
ipy_display(show_filter_button, filter_output, print_button, sql_output)
show_filter_button.on_click(print_filters_on_button_clicked)
print_button.on_click(print_sql_on_button_clicked)